In [1]:
from openpyxl import load_workbook
import pandas as pd
import numpy as np
import json

#### Data prep

Meant to be run from a directory with the following structure:
```
current/
  |- clean/
  |- PE2021_CVR_Final/
      |- (all the .xlsx files exactly as unzipped)
```

Order of operations:
* convert XLSX to CSV (`clean_and_save_files` which calls `xlsx_to_csv`)
* load in Manhattan files from CSV (`make_county_file`)
* get lookup table for candidate codes (`make_cand_lookup`)
* create file for a specific race (`make_race_file`)

In [162]:
def xlsx_to_csv(key):
    wb = load_workbook('./PE2021_CVR_Final/'+ key +'.xlsx')
    ws = wb.active
    # df = pd.DataFrame(ws.values)
    
    data = ws.values
    cols = next(data)[1:]
    data = list(data)
    idx = [r[0] for r in data]
    data = (islice(r, 1, None) for r in data)
    df = pd.DataFrame(data, index=idx, columns=cols)
    
    # https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/
    # dropping null value columns to avoid errors
    df.dropna(inplace = True)

    # new data frame with split value columns
    new = df["Precinct"].str.split(" ", n = 4, expand = True)

    # # making separate first name column from new data frame
    df["AD"]= new[1]

    # # making separate last name column from new data frame
    df["ED"]= new[3]

    # # Dropping old Name columns
    df.drop(columns=["Precinct", "Ballot Style"], inplace = True)
    
    lead_cols = ['AD', 'ED']
    new_cols = lead_cols + [col for col in df.columns if col not in lead_cols]
    df = df = df[new_cols]
    
    df.to_csv('./clean/'+key+'.csv', index_label='id')

In [45]:
# TODO automate reading of directory
files = [
    "2021P1V1_ELE1",
    "2021P1V1_ELE2",
    "2021P1V1_ELE3",
    "2021P1V1_ABS",
    "2021P1V1_AFF",
    "2021P1V1_EMG",
]
def clean_and_save_files():
    for key in files:
        xlsx_to_csv(key)

In [48]:
# this only needs to be run once, and is time-consuming (~10 mins?)
# clean_and_save_files()

In [2]:
def make_county_file():
    # TODO make dynamic using file_names param
    ele1 = pd.read_csv('./clean/2021P1V1_ELE1.csv')
    ele2 = pd.read_csv('./clean/2021P1V1_ELE2.csv')
    ele3 = pd.read_csv('./clean/2021P1V1_ELE3.csv')
    _abs = pd.read_csv('./clean/2021P1V1_ABS.csv')
    _aff = pd.read_csv('./clean/2021P1V1_AFF.csv')
    _emg = pd.read_csv('./clean/2021P1V1_EMG.csv')
    
    # careful! for some reason this file has a different structure than the rest
    # only get the columns that are also in the other files
    _emg_match = _emg[ele1.columns]
    
    # sanity check: how many rows are in all of these files?
    print(ele1.shape[0] + ele2.shape[0] + ele3.shape[0] + _abs.shape[0] + _aff.shape[0] + _emg.shape[0])
    
    df = ele1.append(ele2).append(ele3).append(_abs).append(_aff).append(_emg_match)
    return df

In [3]:
df = make_county_file()

274806


In [4]:
def make_cand_lookup():
    cands = pd.read_csv('./clean/2021P_CandidacyID_To_Name.csv')
    cand_ids = list(cands["CandidacyID"])
    cand_names = list(cands["DefaultBallotName"])
    cand_lookup = { str(id): cand_names[i] for i,id in enumerate(cand_ids) }
    
    # undervotes get skipped in the counting, but overvotes disqualify the vote in that race
    cand_lookup["undervote"] = "" 
#     cand_lookup["overvote"] = ""

    return cand_lookup

In [5]:
cand_lookup = make_cand_lookup()

In [22]:
# hard-coded values for Manhattan
# TODO make better data structure
col_lookup = {
    'bp': df.columns[3:8],
    'comp': df.columns[8:13],
    'cc2': df.columns[13:16],
    'cc10': df.columns[16:21],
    'cc1': df.columns[21:26],
    'cc3': df.columns[26:31],
    'cc5': df.columns[31:36],
    'cc6': df.columns[36:41],
    'cc7': df.columns[41:46],
    'cc8': df.columns[46:51],
    'cc9': df.columns[51:56],
    'mayor': df.columns[56:61],
    'pa': df.columns[61:65]
}
always_cols = ["id", "AD", "ED"]

def make_race_file(df, key):
    cols = list(col_lookup[key])

    # filter full table on votes where the first choice is not empty
    # this is a heuristic: at least one valid vote for a candidate in this race
    race_df = df[(df[cols[0]] != 'undervote')
                 | (df[cols[1]] != 'undervote')
                 | (df[cols[2]] != 'undervote')
                 | (df[cols[3]] != 'undervote')
                 | (df[cols[4]] != 'undervote')
                ][cols + always_cols]
    
    v_cols = always_cols
    for i,_v in enumerate(cols):
        v_col = "v"+str(i+1)
        race_df[v_col] = race_df[cols[i]].replace(cand_lookup)
        v_cols.append(v_col)
        
    return race_df[v_cols]

In [29]:
race_key = 'cc9'
race_df = make_race_file(df, race_key)
race_df

In [ ]:
# race_df.to_csv('./clean/'+race_key+'.csv',index=False)

In [24]:
def keepdupe(row):
    new_row = []
    for r in row:
        if r != '':
            new_row.append(r)
    return ",".join(new_row)

def dedupe(row):
    new_row = []
    for r in row:
#         if r == 'overvote':
#             return ",".join(new_row)
        if r != '' and r not in new_row:
            new_row.append(r)
    return ",".join(new_row)

def make_agg_file(orig, should_dedupe=False):
    agg_df = orig.copy()
    
    if should_dedupe:
        agg_df["v"] = agg_df[["v1","v2","v3","v4","v5"]].apply(dedupe, axis=1)
    else:
        agg_df["v"] = agg_df[["v1","v2","v3","v4","v5"]].apply(keepdupe, axis=1)
    
    agg_grouped = agg_df.groupby("v")["v"].count().reset_index(name="count")

    vs = agg_grouped["v"].str.split(",", expand = True)

    agg_grouped["v1"] = vs[0]
    agg_grouped["v2"] = vs[1]
    agg_grouped["v3"] = vs[2]
    agg_grouped["v4"] = vs[3]
    agg_grouped["v5"] = vs[4]
    agg_grouped.fillna("", inplace=True)

    return agg_grouped

In [33]:
agg_grouped = make_agg_file(race_df, should_dedupe=True)
agg_grouped

,v,count,v1,v2,v3,v4,v5
0,Athena Moore,426,Athena Moore,,,,
1,"Athena Moore,Bernadette McNear",10,Athena Moore,Bernadette McNear,,,
2,"Athena Moore,Bernadette McNear,Bill Perkins",4,Athena Moore,Bernadette McNear,Bill Perkins,,
3,"Athena Moore,Bernadette McNear,Bill Perkins,Co...",1,Athena Moore,Bernadette McNear,Bill Perkins,Cordell Cleare,Joshua Albert Clennon
4,"Athena Moore,Bernadette McNear,Bill Perkins,Ke...",1,Athena Moore,Bernadette McNear,Bill Perkins,Keith Taylor,Joshua Albert Clennon
...,...,...,...,...,...,...,...
11669,"overvote,William A. Allen",3,overvote,William A. Allen,,,
11670,"overvote,William A. Allen,Athena Moore,Sheba T...",1,overvote,William A. Allen,Athena Moore,Sheba T. Simpson-Amsterdam,Bill Perkins
11671,"overvote,William A. Allen,Keith Taylor",1,overvote,William A. Allen,Keith Taylor,,
11672,"overvote,William A. Allen,Pierre A. Gooding,At...",1,overvote,William A. Allen,Pierre A. Gooding,Athena Moore,Joshua Albert Clennon


In [30]:
# agg_grouped.to_csv('./clean/'+race_key+'_grouped_dedupe.csv', index=False)

#### Analysis

In [34]:
# QA first-round votes
agg_grouped.groupby("v1").sum("count")

,count
v1,
Athena Moore,2802
Bernadette McNear,334
Bill Perkins,5409
Billy Council,736
Cordell Cleare,2544
Joshua Albert Clennon,1389
Keith Taylor,1432
Kristin Richardson Jordan,4873
Mario Rosser,2176


In [35]:
# QA round 2 transfers from Write-in to candidates
# note: this does not account for multiple write-ins followed by a "real" candidate
agg_grouped[(agg_grouped["v1"] == "Write-in")].groupby("v2").sum("count")

,count
v2,
,24
Athena Moore,4
Bill Perkins,6
Cordell Cleare,3
Joshua Albert Clennon,1
Keith Taylor,1
Kristin Richardson Jordan,2
Mario Rosser,2
Pierre A. Gooding,2


In [38]:
ret = []
abbrevs = {
    'overvote': 'o',
    'Write-in': 'I',
    
    # bp
    'Ben Kallos': 'K',
    'Brad M. Hoylman': 'H',
    'Elizabeth R. Caputo': 'C',
    'Guillermo A. Perez': 'P',
    'Kimberly R. Watkins': 'W',
    'Lindsey C. Boylan': 'B',
    'Mark D. Levine': 'L',
    
    # cc7
    'Carmen R. Quinones': 'Q',
    'Corey Ortega': 'R',
    'Daniel M. Cohen': 'C',
    'Keith L. Harris': 'H',
    'Lena Melendez': 'D',
    'Luis Tejada': 'T',
    'Maria Ordonez': 'O',
    'Marti Gould  Allen-Cummings': 'M',
    'Miguel Estrella': 'E',
    'Raymond Sanchez Jr.': 'S',
    'Shaun Abreu': 'A',
    'Stacy R. Lynch': 'L',
    
    # cc6
    'David Gold': 'G',
    'Gale A. Brewer': 'B',
    'Jeffrey Omura': 'O',
    'Maria Danzilo': 'D',
    'Sara Lind': 'L',
    'Zachary Tov Weiner': 'W',
    
    # cc9
    'Bill Perkins': 'P',
    'Kristin Richardson Jordan': 'J',
    'Athena Moore': 'M',
    'Cordell Cleare': 'C',
    'Mario Rosser': 'R',
    'William A. Allen': 'A',
    'Keith Taylor': 'T',
    'Joshua Albert Clennon': 'L',
    'Ruth L. McDaniels': 'D',
    'Billy Council': 'O',
    'Pierre A. Gooding': 'G',
    'Sheba T. Simpson-Amsterdam': 'S',
    'Bernadette McNear': 'N'
}
for i,v in enumerate(agg_grouped["v"].str.split(",", expand=False)):
    ret.append({ "v": [abbrevs[e] for e in v], "votes": int(agg_grouped["count"][i]) })
ret

[{'v': ['M'], 'votes': 426},
 {'v': ['M', 'N'], 'votes': 10},
 {'v': ['M', 'N', 'P'], 'votes': 4},
 {'v': ['M', 'N', 'P', 'C', 'L'], 'votes': 1},
 {'v': ['M', 'N', 'P', 'T', 'L'], 'votes': 1},
 {'v': ['M', 'N', 'P', 'D', 'J'], 'votes': 1},
 {'v': ['M', 'N', 'P', 'S', 'L'], 'votes': 1},
 {'v': ['M', 'N', 'P', 'S', 'D'], 'votes': 1},
 {'v': ['M', 'N', 'P', 'A', 'T'], 'votes': 1},
 {'v': ['M', 'N', 'O', 'G', 'D'], 'votes': 1},
 {'v': ['M', 'N', 'O', 'S'], 'votes': 1},
 {'v': ['M', 'N', 'C'], 'votes': 1},
 {'v': ['M', 'N', 'C', 'P'], 'votes': 1},
 {'v': ['M', 'N', 'C', 'P', 'D'], 'votes': 1},
 {'v': ['M', 'N', 'C', 'G', 'J'], 'votes': 1},
 {'v': ['M', 'N', 'C', 'G', 'S'], 'votes': 1},
 {'v': ['M', 'N', 'C', 'D', 'P'], 'votes': 1},
 {'v': ['M', 'N', 'C', 'S'], 'votes': 1},
 {'v': ['M', 'N', 'C', 'S', 'P'], 'votes': 1},
 {'v': ['M', 'N', 'L', 'A', 'O'], 'votes': 1},
 {'v': ['M', 'N', 'T', 'P', 'G'], 'votes': 2},
 {'v': ['M', 'N', 'T', 'P', 'D'], 'votes': 2},
 {'v': ['M', 'N', 'T', 'O', 'D'],

In [39]:
# with open('./clean/agg_'+race_key+'_dedupe.json', 'w') as f:
#     json.dump(ret, f, indent=2)